## Lab No 3: Entity Resolution Application using PySpark

1) Develop a PySpark script to clean and preprocess data before performing entity resolution. 
Include steps like tokenization and normalization.

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark=SparkSession.builder.appName("Entity").getOrCreate()

In [39]:
df=spark.read.csv('/home/5AIMLA2/Documents/230962322/BDA-LAB/customer.csv',header=True,inferSchema=True,sep=",")


df1=df.dropna()
cols=df1.columns
print(cols)
for i in cols:
    df1=df1.withColumn(i,lower(trim(df1[i])))

df1.show(100)



['id', 'name', 'city']
+---+-------------+-----------+
| id|         name|       city|
+---+-------------+-----------+
|  1|     john doe|   new york|
|  2|      jon doe|   new york|
|  3|   johnny doe|los angeles|
|  4|   jane smith|    chicago|
|  5|  janet smith|    chicago|
|  6|     john doe|   new york|
|  7| jonathon doe|   new york|
|  8|       jon do|   new york|
|  9|      john d.|   new york|
| 10|     john doe|los angeles|
| 11|   jane smyth|    chicago|
| 12|     j. smith|    chicago|
| 13| janet smythe|    chicago|
| 14|  janie smith|    chicago|
| 15|    johnny do|los angeles|
| 16|johnathan doe|     boston|
| 17|     john doe|     boston|
| 18|  john a. doe|   new york|
| 19| john doe jr.|   new york|
| 20|   johnny doe|     newark|
| 21|      jon doe|los angeles|
| 22|   jane smith|los angeles|
| 23|     j. smyth|    chicago|
| 24|      jane s.|    chicago|
| 25|  janet smith|     boston|
| 26|  janet smith|   new york|
| 27|     john doe|    chicago|
| 28|     john do

2) Implement a PySpark program that computes similarity scores between records using a 
chosen similarity metric.

In [40]:
df2=df1
df2.show()

+---+-------------+-----------+
| id|         name|       city|
+---+-------------+-----------+
|  1|     john doe|   new york|
|  2|      jon doe|   new york|
|  3|   johnny doe|los angeles|
|  4|   jane smith|    chicago|
|  5|  janet smith|    chicago|
|  6|     john doe|   new york|
|  7| jonathon doe|   new york|
|  8|       jon do|   new york|
|  9|      john d.|   new york|
| 10|     john doe|los angeles|
| 11|   jane smyth|    chicago|
| 12|     j. smith|    chicago|
| 13| janet smythe|    chicago|
| 14|  janie smith|    chicago|
| 15|    johnny do|los angeles|
| 16|johnathan doe|     boston|
| 17|     john doe|     boston|
| 18|  john a. doe|   new york|
| 19| john doe jr.|   new york|
| 20|   johnny doe|     newark|
+---+-------------+-----------+
only showing top 20 rows


In [57]:
df1=df1.withColumn('block_key',substring('name',1,1)) #ie all of them will be compared
df1.show(100)
df2.show()
df2=df2.withColumnRenamed('name','name2')


+---+-------------+-----------+---------+
| id|         name|       city|block_key|
+---+-------------+-----------+---------+
|  1|     john doe|   new york|        j|
|  2|      jon doe|   new york|        j|
|  3|   johnny doe|los angeles|        j|
|  4|   jane smith|    chicago|        j|
|  5|  janet smith|    chicago|        j|
|  6|     john doe|   new york|        j|
|  7| jonathon doe|   new york|        j|
|  8|       jon do|   new york|        j|
|  9|      john d.|   new york|        j|
| 10|     john doe|los angeles|        j|
| 11|   jane smyth|    chicago|        j|
| 12|     j. smith|    chicago|        j|
| 13| janet smythe|    chicago|        j|
| 14|  janie smith|    chicago|        j|
| 15|    johnny do|los angeles|        j|
| 16|johnathan doe|     boston|        j|
| 17|     john doe|     boston|        j|
| 18|  john a. doe|   new york|        j|
| 19| john doe jr.|   new york|        j|
| 20|   johnny doe|     newark|        j|
| 21|      jon doe|los angeles|   

In [60]:
df_joined=df1.alias('df1').join(df1.alias('df2'), on='block_key').where(col('df1.id')<col('df2.id'))
df_joined.show(2500)

+---------+---+-------------+-----------+---+-------------+-----------+
|block_key| id|         name|       city| id|         name|       city|
+---------+---+-------------+-----------+---+-------------+-----------+
|        j|  1|     john doe|   new york| 50|    john doe#|     newark|
|        j|  1|     john doe|   new york| 49|    john doe@|    chicago|
|        j|  1|     john doe|   new york| 48|  janet sm‡th|   new york|
|        j|  1|     john doe|   new york| 47|  jane† smith|     boston|
|        j|  1|     john doe|   new york| 46|     jane s.™|    chicago|
|        j|  1|     john doe|   new york| 45|     j. sm¥th|    chicago|
|        j|  1|     john doe|   new york| 44|   jane sm&th|los angeles|
|        j|  1|     john doe|   new york| 43|     jon% doe|los angeles|
|        j|  1|     john doe|   new york| 42|  johnny d()e|     newark|
|        j|  1|     john doe|   new york| 41|    john doe!|   new york|
|        j|  1|     john doe|   new york| 40|  john a. d$e|   ne

In [ ]:
df_result=df_joined.withColumn('lev_distance',levenshtein(col('name'),col('name2')))
df_result.show()

+---+--------+--------+---------+---+-------------+-----------+------------+
| id|    name|    city|block_key| id|        name2|       city|lev_distance|
+---+--------+--------+---------+---+-------------+-----------+------------+
|  1|john doe|new york|     NULL|  1|     john doe|   new york|           0|
|  1|john doe|new york|     NULL|  2|      jon doe|   new york|           1|
|  1|john doe|new york|     NULL|  3|   johnny doe|los angeles|           2|
|  1|john doe|new york|     NULL|  4|   jane smith|    chicago|           8|
|  1|john doe|new york|     NULL|  5|  janet smith|    chicago|           9|
|  1|john doe|new york|     NULL|  6|     john doe|   new york|           0|
|  1|john doe|new york|     NULL|  7| jonathon doe|   new york|           4|
|  1|john doe|new york|     NULL|  8|       jon do|   new york|           2|
|  1|john doe|new york|     NULL|  9|      john d.|   new york|           2|
|  1|john doe|new york|     NULL| 10|     john doe|los angeles|           0|

In [61]:
df_result.filter(df_result['name']!=df_result['name2'])

#compute similarity score for the names

df_result=df_result.withColumn('max_len',greatest(length(col('name')),length(col('name2')))) \
          .withColumn('similarity_score',1-col('lev_distance')/col('max_len'))

        
df_result.show()

+---+--------+--------+---------+---+-------------+-----------+------------+-------+-------------------+
| id|    name|    city|block_key| id|        name2|       city|lev_distance|max_len|   similarity_score|
+---+--------+--------+---------+---+-------------+-----------+------------+-------+-------------------+
|  1|john doe|new york|     NULL|  1|     john doe|   new york|           0|      8|                1.0|
|  1|john doe|new york|     NULL|  2|      jon doe|   new york|           1|      8|              0.875|
|  1|john doe|new york|     NULL|  3|   johnny doe|los angeles|           2|     10|                0.8|
|  1|john doe|new york|     NULL|  4|   jane smith|    chicago|           8|     10|0.19999999999999996|
|  1|john doe|new york|     NULL|  5|  janet smith|    chicago|           9|     11|0.18181818181818177|
|  1|john doe|new york|     NULL|  6|     john doe|   new york|           0|      8|                1.0|
|  1|john doe|new york|     NULL|  7| jonathon doe|   n